1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다
    - 토큰 수 초과로 답변을 생성하지 못할 수 있음
    - 문서가 길면(인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 => 벡터 데이터베이스에 저장
4. 질문이 있을 때, 백터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

# 1. 문서의 내용을 읽는다.

In [ ]:
%pip install python-docx

In [ ]:
from docx import Document

document = Document('./tax.docx')
print(f'document == {document}')


In [ ]:

# document는 paragraphs 형태로 읽음
print(f'document == {dir(document)}')
full_text = ''
for index, paragraph in enumerate(document.paragraphs):
    print(f'paragraph == {paragraph.text}')
    full_text += f'{paragraph.text}\n'

In [ ]:
full_text

# 2. 문서를 쪼갠다.

In [ ]:
%pip install tiktoken

In [ ]:
import tiktoken

encoder = tiktoken.encoding_for_model("gpt-4o")
encoding= encoder.encode(full_text)

In [ ]:
len(encoding)

In [ ]:
decoded = encoder.decode(encoding)

In [ ]:
decoded

In [ ]:
import tiktoken

def split_text(full_text, chunk_size):
    encoder = tiktoken.encoding_for_model("gpt-4o")
    total_encoding = encoder.encode(full_text)
    total_token_count = len(total_encoding)
    text_list = []
    for i in range(0, total_token_count, chunk_size):
        chunk = total_encoding[i: i + chunk_size]
        decoded = encoder.decode(chunk)
        text_list.append(decoded)
    return text_list

In [ ]:
chunk_list = split_text(full_text, 1500)

In [ ]:
chunk_list

# 3. 문서 임베딩

In [ ]:
%pip install chromadb

In [ ]:
import chromadb

chroma_client = chromadb.Client()

In [ ]:
# collection => table과 비슷한 개념
collection_name = 'tax_collection'
try:
    chroma_client.delete_collection(collection_name)
    print(f"기존 컬렉션 '{collection_name}' 삭제됨")
except Exception as e:
    print(f"컬렉션 삭제 중 오류: {e}")
tax_collection = chroma_client.create_collection(collection_name)

In [ ]:
%pip install openai

In [ ]:
import os
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_embedding = OpenAIEmbeddingFunction(api_key=openai_api_key,model_name="text-embedding-3-large")

In [ ]:
try:
    chroma_client.delete_collection(collection_name)
    print(f"기존 컬렉션 '{collection_name}' 삭제됨")
except Exception as e:
    print(f"컬렉션 삭제 중 오류: {e}")
    
tax_collection = chroma_client.get_or_create_collection(collection_name, embedding_function=openai_embedding)

In [ ]:
tax_collection

In [ ]:
id_list = []
for index in range(len(chunk_list)):
    id_list.append(f'{index}')

In [ ]:
len(id_list)

In [ ]:
len(chunk_list)

In [ ]:
tax_collection.add(documents=chunk_list, ids=id_list)

# 4. 유사도 검색

In [ ]:
query='연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_doc = tax_collection.query(query_texts=query)
retrieved_doc

In [ ]:
retrieved_doc['documents'][0]

# 5. LLM 질의

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": f"당신은 한국의 소득세 전문가입니다. 아래 내용을 참고하여 사용자의 질문에 답변해주세요 {retrieved_doc['documents'][0]}"},
        {"role": "user", "content": query}
    ]
)

In [ ]:
response

In [ ]:
response.choices[0].message.content